In [4]:
import serial  # 引用pySerial模組
import time
import numpy as np
from time import sleep
import socketio

BAUD_RATE = 115200    # 設定傳輸速率
COM_PORT = 'COM3'    # 指定通訊埠名稱
CHANNEL_NUMBER = 3
SAMPLE_TIME = 1 / 500
WINDOW_SIZE = 300

record_path = './data/lick2.npy'
server_url = 'http://localhost:3000'

In [2]:
class reciever():
    def __init__(self, url, port, baud_rate, record = None) -> None: 
        super().__init__()
        if isinstance(record, loader):
            self._serial = record
        else:
            self._serial = serial.Serial(port, baud_rate)# 初始化序列通訊埠
        self._sio = socketio.Client()
        self._sio.connect(url)
        try:
            assert isinstance(self._sio, socketio.Client)
            print("Connected to Socket server.")
        except:
            raise Exception("Maybe server is not online.")
    
    def run(self):
        try:
            while True:
                while self._serial.in_waiting:          # 若收到序列資料…
                    data_raw = self._serial.readline()  # 讀取一行
                    rcv = data_raw.decode(errors='surrogateescape').rstrip() # 用預設的UTF-8解碼
                    try:
                        self._sio.emit("ArduinoSignal", rcv)
                        print(rcv)
                    except KeyboardInterrupt:
                        break
                    except:
                        pass
                    finally:
                        if isinstance(self._serial, loader):
                            sleep(SAMPLE_TIME) 

        except KeyboardInterrupt:
            self._serial.close()    # 清除序列通訊物件
            self._sio.disconnect()
            print('Serial disconnected.')
        
            
class loader():
    def __init__(self, path) -> None:
        self.data = np.load(path)
        self.in_waiting = True
        self.__i = 0
        
    def readline(self):
        try:
            ret = self.data[self.__i, :].astype(np.str_).tolist()
            return ",".join(ret).encode(encoding='UTF-8')
        except KeyboardInterrupt:
            self.in_waiting = False
        except:
            self.__i = 0
            ret = self.data[self.__i, :].astype(np.str_).tolist()
            return ",".join(ret).encode(encoding='UTF-8')
        finally:
            self.__i += 1
    
    def close(self):
        pass

In [ ]:
arduino = reciever(server_url, COM_PORT, BAUD_RATE, loader(record_path))
arduino.run()